## Models

**Random Forest**

In [ ]:
m = rf(xs, y, n_estimators=1000,  criterion='entropy', class_weight='balanced')
m_roc(y, xs), m_roc(valid_y, valid_xs)

**Neural Network**

**Gradient Boosted Tree**

In [ ]:
%%time
clf = HistGradientBoostingClassifier(loss='binary_crossentropy', verbose=1, l2_regularization=2.4,
                             learning_rate=0.03, max_depth=27, max_iter=1000, max_leaf_nodes=44,
                             min_samples_leaf=8,  scoring='roc_auc', tol=1e-8)
clf.fit(xs, y)
m_roc(y, xs), m_roc(valid_y, valid_xs)